In [113]:
import os
import torch
from torch.utils.data import DataLoader, TensorDataset
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModel
from tqdm.notebook import trange, tqdm
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from info_nce import InfoNCE
import pyarrow as pa

### Training functions

In [85]:
class CustomDataset(TensorDataset):

    def __init__(self, dataframe, tokenizer):
        self.tokenizer = tokenizer
        self.doc = dataframe["Comment"]
        self.code = dataframe["Code"]
        # self.targets = dataframe.labels
        self.max_len = 256

    def __len__(self):
        assert len(self.doc) == len(self.code)
        return len(self.doc)

    def __getitem__(self, index):
        doc = str(self.doc[index])
        doc = " ".join(doc.split())

        code = str(self.code[index])
        doc_inputs = self.tokenizer.encode_plus(
            doc,
            add_special_tokens=False,
            max_length=self.max_len,
            truncation=True,
            padding='max_length',
            return_token_type_ids=False
        )
        doc_ids = doc_inputs['input_ids']
        doc_mask = doc_inputs['attention_mask']

        code_inputs = self.tokenizer.encode_plus(
            code,
            add_special_tokens=False,
            max_length=self.max_len,
            truncation=True,
            padding='max_length',
            return_token_type_ids=False
        )

        code_ids = code_inputs['input_ids']
        code_mask = code_inputs['attention_mask']

        return {
            'doc_ids': torch.tensor(doc_ids, dtype=torch.long),
            'doc_mask': torch.tensor(doc_mask, dtype=torch.long),
            'code_ids': torch.tensor(code_ids, dtype=torch.long),
            'code_mask': torch.tensor(code_mask, dtype=torch.long),
        }

def train(model, optimizer, training_set, epochs):
    print("Start training")
    model.train()
    train_dataloader = DataLoader(training_set, batch_size=32, shuffle=True)
    for epoch in tqdm(range(1, epochs + 1)):

        all_losses = []   
        progress_bar = tqdm(train_dataloader, desc=f'Epoch {epoch}')
        
        for idx, batch in enumerate(progress_bar):

            if batch['code_ids'].size(0) < 32:
                continue

            query_id = batch['doc_ids'].to(device)#.unsqueeze(0)
            query_mask = batch['doc_mask'].to(device)#.unsqueeze(0)
            inputs = {'input_ids': query_id[0,:].unsqueeze(0), 'attention_mask': query_mask[0,:].unsqueeze(0)}
            
            query = model(**inputs)
            #print("query shape: ", query[0].shape)
            #query = query.last_hidden_state.squeeze(0).mean(dim=0)
            #print("query shape: ", query.shape)
            
            code_ids = batch['code_ids'].to(device)
            code_masks = batch['code_mask'].to(device)
            

            inputs = {'input_ids': code_ids[0,:].unsqueeze(0), 'attention_mask': code_masks[0,:].unsqueeze(0)}
            positive_code_key = model(**inputs)
            #positive_code_key = positive_code_key.last_hidden_state.squeeze(0).mean(dim=0)
            #print("p code", positive_code_key.shape)
            

            inputs = {'input_ids': code_ids[1:], 'attention_mask': code_masks[1:]}
            negative_code_keys = model(**inputs)
            #negative_code_keys = negative_code_keys.last_hidden_state.mean(dim=1)
            #print("n code", negative_code_keys.shape)
            
            #break
            
            loss = info_nce_loss(query, positive_code_key, negative_code_keys)
            
            loss.backward()
            
            all_losses.append(loss.to("cpu").detach().numpy())

            if (idx + 1) % 2 == 0:
                optimizer.step()
                optimizer.zero_grad()
                      
        train_mean_loss = np.mean(all_losses)
        print(f'Epoch {epoch} - Train-Loss: {train_mean_loss}')

def info_nce_loss(query, positive_key, negative_keys):
    query = torch.flatten(query).view(-1).unsqueeze(0)
    positive_key = torch.flatten(positive_key).view(-1).unsqueeze(0)
    negative_keys = torch.stack([torch.flatten(code_emb).view(-1) for code_emb in negative_keys])
    return InfoNCE(negative_mode='unpaired')(query, positive_key, negative_keys)

### Eval functions

In [86]:
def create_embs(test_set, model):

    tokenizer = AutoTokenizer.from_pretrained("Salesforce/codet5p-110m-embedding", trust_remote_code=True)
    
    model.to(device)
    
    representations = []
    # Process each input separately
    for input_text in test_set:
        # Tokenize without padding
        encoded_input = tokenizer(input_text, padding=False, truncation=True, max_length=256, return_tensors="pt")
        
        # Move input to the same device as the model
        encoded_input = {k: v.to(model.device) for k, v in encoded_input.items()}
        
        # Get model output
        with torch.no_grad():
            #output = model.encoder(**encoded_input) # For CodeT5+ last hidden layer
            output = model(**encoded_input) # Standard forwardin
        
        # Add to representations list
        representations.append(output.cpu().numpy())

    return np.array(representations)

def predict_distances(doc_embs, code_embs):
    all_distances = []
    progress_bar = tqdm(range(len(doc_embs)), desc="Compute Distances")
    for idx in progress_bar:
        # Sample Query
        query = doc_embs[idx]
        query = np.expand_dims(query, axis=0)
        
        # Sample Positive Code
        positive_code = code_embs[idx]
        positive_code = np.expand_dims(positive_code, axis=0)
        
        # Calculate Cosine distance for positive code
        positive_distance = calculate_cosine_distance(query, positive_code)
        
        # Calculate distances for all negative codes
        negative_distances = []
        for neg_idx, negative_code in enumerate(code_embs):
            if neg_idx != idx:  # Exclude the positive pair
                negative_code = np.expand_dims(negative_code, axis=0)
                distance = calculate_cosine_distance(query, negative_code)
                negative_distances.append(distance)
        
        # Combine positive and negative distances
        distances = [positive_distance] + negative_distances
        all_distances.append(distances)

    return all_distances

def calculate_mrr_from_distances(distances_lists):
    ranks = []
    for batch_idx, predictions in enumerate(distances_lists):
        correct_score = predictions[0]
        scores = np.array([prediction for prediction in predictions])
        rank = np.sum(scores <= correct_score)
        ranks.append(rank)
    mean_mrr = np.mean(1.0 / np.array(ranks))

    return mean_mrr

def calculate_cosine_distance(code_key, query):
    code_key = np.squeeze(code_key)
    query = np.squeeze(query)
    
    cosine_similarity = (np.dot(query, code_key) /
                         (np.linalg.norm(query) * np.linalg.norm(code_key)))
    # Compute cosine distance
    return 1 - cosine_similarity

### Execute

In [87]:
ds = load_dataset("drndr/statcodesearch")

In [88]:
tokenizer = AutoTokenizer.from_pretrained("Salesforce/codet5p-110m-embedding", trust_remote_code=True)
dataset = ds["test"]

# Split the dataset into train and test sets with an 80/20 ratio
train_data, test_data = train_test_split(
    dataset, 
    test_size=0.2,    # 20% for testing
    train_size=0.8,   # 80% for training
    random_state=42   # For reproducible results
)

training_set = CustomDataset(train_data, tokenizer)
test_set = CustomDataset(test_data, tokenizer)


In [96]:
model = AutoModel.from_pretrained("Salesforce/codet5p-110m-embedding", trust_remote_code=True)
optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-5)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

CodeT5pEmbeddingModel(
  (shared): Embedding(32103, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32103, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dropout(

### Pre-trained eval

In [90]:
code_embs = create_embs(test_data["Code"], model)
doc_embs = create_embs(test_data["Comment"], model)
all_distances = predict_distances(doc_embs, code_embs)
mrr = calculate_mrr_from_distances(all_distances)
print(mrr)

Compute Distances:   0%|          | 0/214 [00:00<?, ?it/s]

0.5577294425478596


In [ ]:
all_code_embs = create_embs(ds["test"]["Code"], model)
all_doc_embs = create_embs(ds["test"]["Comment"], model)
all_distances = predict_distances(all_doc_embs, all_code_embs)
mrr = calculate_mrr_from_distances(all_distances)
print(mrr)

### Fine-tuned eval

In [97]:
train(model, optimizer, training_set, 100)

Start training


  0%|          | 0/100 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/27 [00:00<?, ?it/s]

Epoch 1 - Train-Loss: 3.333803653717041


Epoch 2:   0%|          | 0/27 [00:00<?, ?it/s]

...

Epoch 98 - Train-Loss: 0.8757592439651489


Epoch 99:   0%|          | 0/27 [00:00<?, ?it/s]

Epoch 99 - Train-Loss: 0.5764246582984924


Epoch 100:   0%|          | 0/27 [00:00<?, ?it/s]

Epoch 100 - Train-Loss: 0.8005322217941284


In [99]:
code_embs = create_embs(test_data["Code"], model)
doc_embs = create_embs(test_data["Comment"], model)
all_distances = predict_distances(doc_embs, code_embs)
mrr = calculate_mrr_from_distances(all_distances)
print(mrr)

Compute Distances:   0%|          | 0/214 [00:00<?, ?it/s]

0.6035769910504865


In [104]:
all_code_embs = create_embs(ds["test"]["Code"], model)
all_doc_embs = create_embs(ds["test"]["Comment"], model)
all_distances = predict_distances(all_doc_embs, all_code_embs)
mrr = calculate_mrr_from_distances(all_distances)
print(mrr)

Compute Distances:   0%|          | 0/1070 [00:00<?, ?it/s]

0.5372501951146463


In [116]:
df = pd.DataFrame(np.squeeze(all_code_embs, axis=1))
# Create Hugging Face Dataset
hf_dataset = Dataset.from_pandas(df)
# Save to disk
hf_dataset.save_to_disk("embeddings_code")

Saving the dataset (0/1 shards):   0%|          | 0/1070 [00:00<?, ? examples/s]

In [117]:
df = pd.DataFrame(np.squeeze(all_doc_embs, axis=1))
# Create Hugging Face Dataset
hf_dataset = Dataset.from_pandas(df)
# Save to disk
hf_dataset.save_to_disk("embeddings_doc")

Saving the dataset (0/1 shards):   0%|          | 0/1070 [00:00<?, ? examples/s]